# Libraries to import

In [ ]:
# libraries to import
import pandas as pd
import matplotlib.pyplot as plt
import csv
import os
import io
from imutils import build_montages
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2
from IPython.display import HTML
import base64 

# Recipes.zip

In [ ]:
print("Current Working Directory " , os.getcwd())
os.chdir("C:\\Users\prern\Downloads")

import zipfile
from PIL import Image

imgzip = zipfile.ZipFile("recipes.zip")
inflist = imgzip.infolist()

print(inflist)
    # display(img)

Current Working Directory  C:\Users\prern
[<ZipInfo filename='recipes/' external_attr=0x10>, <ZipInfo filename='recipes/CEteOEzpt5e.png' external_attr=0x20 file_size=530794>, <ZipInfo filename='recipes/CF4Y-Zmlz78.png' external_attr=0x20 file_size=460371>, <ZipInfo filename='recipes/CF66vPaJAac.png' external_attr=0x20 file_size=764254>, <ZipInfo filename='recipes/CFchW_9pOQT.png' external_attr=0x20 file_size=582290>, <ZipInfo filename='recipes/CFCP82BJkZ8.png' compress_type=deflate external_attr=0x20 file_size=527444 compress_size=527031>, <ZipInfo filename='recipes/CFfLCriphU6.png' compress_type=deflate external_attr=0x20 file_size=660726 compress_size=658685>, <ZipInfo filename='recipes/CFg-xYrDksw.png' external_attr=0x20 file_size=555225>, <ZipInfo filename='recipes/CFhhOROj_bo.png' external_attr=0x20 file_size=553716>, <ZipInfo filename='recipes/CFhW7r5phjp.png' external_attr=0x20 file_size=459644>, <ZipInfo filename='recipes/CFjg8ZAjF9G.png' external_attr=0x20 file_size=555519>, <

In [ ]:
z = zipfile.ZipFile('recipes.zip')
for i in range(len(z.namelist())):

    file_in_zip = z.namelist()[i]
    if (".png" in file_in_zip or ".PNG" in file_in_zip):

        data = z.read(file_in_zip)
        dataEnc = io.BytesIO(data)
        img = Image.open(dataEnc)
        

In [ ]:
# importing required modules
from zipfile import ZipFile
  
# specifying the zip file name
file_name = "recipes.zip"
  
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
  
    # extracting all the files
    print('Extracting all the files now...')
    a=zip.extractall()
    print('Done!')
    
import glob
pngfiles = []
for file in glob.glob("C:/Users/prern/Downloads/recipes/*.png"):
    pngfiles.append(file)
 

# RGB Score

In [ ]:
def image_colorfulness(image):
	# split the image into its respective RGB components
	(B, G, R) = cv2.split(image.astype("float"))
	# compute rg = R - G
	rg = np.absolute(R - G)
	# compute yb = 0.5 * (R + G) - B
	yb = np.absolute(0.5 * (R + G) - B)
	# compute the mean and standard deviation of both `rg` and `yb`
	(rbMean, rbStd) = (np.mean(rg), np.std(rg))
	(ybMean, ybStd) = (np.mean(yb), np.std(yb))
	# combine the mean and standard deviations
	stdRoot = np.sqrt((rbStd ** 2) + (ybStd ** 2))
	meanRoot = np.sqrt((rbMean ** 2) + (ybMean ** 2))
	# derive the "colorfulness" metric and return it
	return stdRoot + (0.3 * meanRoot)

In [ ]:
# remove corrupt file
pngfiles.remove('C:/Users/prern/Downloads/recipes\\CMmV2A2sBvY.png')
len(pngfiles)

11695

In [ ]:
# initialize the results list
print("[INFO] computing colorfulness metric for dataset...")
results = []
#test=0
# loop over the image paths


for i in pngfiles:
    # load the image, resize it (to speed up computation), and
    # compute the colorfulness metric for the image
        image = cv2.imread(i)
        print(i)
        Shape=image.shape[1::-1]
        image = imutils.resize(image, width=250)
        C = image_colorfulness(image)
    # display the colorfulness score on the image
        cv2.putText(image, "{:.2f}".format(C), (40, 40), 
        cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 255, 0), 3)
    # add the image and colorfulness metric to the results list
        results.append((image, C, i, Shape))


In [ ]:
print("[INFO] displaying results...")
results = sorted(results, key=lambda x: x[1], reverse=True)
mostColor = [r[0] for r in results[:25]]
leastColor = [r[0] for r in results[-25:]][::-1]

# construct the montages for the two sets of images
mostColorMontage = build_montages(mostColor, (128, 128), (5, 5))
leastColorMontage = build_montages(leastColor, (128, 128), (5, 5))

# display the images
cv2.imshow("Most Colorful", mostColorMontage[0])
cv2.imshow("Least Colorful", leastColorMontage[0])
cv2.waitKey(0)

[INFO] displaying results...


-1

In [ ]:
results = np.array(results, dtype="object")
c=results[:,1:4][:,:]

In [ ]:
Median=np.median(c[:,0:1][:])
Max=np.max(c[:,0:1][:])
Min=np.min(c[:,0:1][:])
Mean=np.mean(c[:,0:1][:])

In [ ]:
# create data frame
c=pd.DataFrame(c)
c.columns=['Color_value','Image_name', 'Shape']

In [ ]:
# download the file
def create_download_link( c, title = "Download CSV file", filename = "data.csv"):  
    csv = c.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(c)